# Simulate all policies
This notebook simulated all candidates policies from 2012-2024.

In [ ]:
# imports
import iot
from iot import iot_user
import numpy as np
import pandas as pd
import json
import plotly.express as px
import os
from plotly.colors import n_colors
from scipy.interpolate import UnivariateSpline
import plotly.graph_objects as go

# set plotly template
template = "plotly_white"
COLOR_SEQUENCE = ["red", "blue"]  # Republican, Democrat
dash_sequence = ['dash', 'dot', 'dashdot', "solid"]  # 2012, 2016, 2020

# Create path for plots to be saved to
CUR_DIR = os.getcwd()
path = os.path.join(CUR_DIR, "plots")
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
# Read in candidate platform JSON files
obama2015_path = "https://raw.githubusercontent.com/jdebacker/examples/pres_proposals/psl_examples/taxcalc/Obama2015.json"
romney2012_path = "https://raw.githubusercontent.com/jdebacker/examples/pres_proposals/psl_examples/taxcalc/Romney2012.json"
clinton2016_path = "https://raw.githubusercontent.com/jdebacker/examples/pres_proposals/psl_examples/taxcalc/Clinton2016.json"
trump2016_path = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/Trump2016.json"
biden2020_path = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/Biden2020.json"
trump2020_path = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/TCJA.json"
harris2024_path = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/Harris2024.json"
trump2024_path = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/Trump2024.json"

pre_2020_baseline = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/2017_law.json"
baseline_2020 = "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/TCJA.json"


candidate_dict = {
    "Obama 2015": {"policy_path": obama2015_path, "baseline_path": [pre_2020_baseline], "start_year": 2016},
    "Romney 2012": {"policy_path": romney2012_path, "baseline_path": [pre_2020_baseline], "start_year": 2014}, #wanted to do 13,  but taxcalc with CPS only goes to 14
    "Clinton 2016": {"policy_path": clinton2016_path, "baseline_path": [pre_2020_baseline], "start_year": 2017},
    "Trump 2016": {"policy_path": trump2016_path, "baseline_path": [pre_2020_baseline], "start_year": 2017},
    "Biden 2020": {"policy_path": biden2020_path, "baseline_path": [pre_2020_baseline, baseline_2020], "start_year": 2021},
    "Trump 2020": {"policy_path": trump2020_path, "baseline_path": [pre_2020_baseline], "start_year": 2021},
    # "Harris 2024": {"policy_path": harris2024_path, "baseline_path": [pre_2020_baseline, baseline_2020], "start_year": 2025},
    # "Trump 2024": {"policy_path": trump2024_path, "baseline_path": [pre_2020_baseline, baseline_2020], "start_year": 2025}
    "Harris 2024": {"policy_path": harris2024_path, "baseline_path": None, "start_year": 2026},  # Use 2026 for these candidates since largest diff is how treat TCJA expirations
    "Trump 2024": {"policy_path": trump2024_path, "baseline_path": None, "start_year": 2026}
    }

In [ ]:
# Create IOT objects for each candidate platform
policies = []
baseline_policies = []
labels = list(candidate_dict.keys())
# get years from start_year in candidate_dict
years = [v["start_year"] for v in candidate_dict.values()]
for k, v in candidate_dict.items():
    # with open(v["policy_path"], "r") as file:
        # json1 = file.read()
    json1 = v["policy_path"]#json.load(open(v["policy_path"]))
    print(json1)
    policies.append(json1)
    if v["baseline_path"] is None:
        json2 = {}
    else:
        for ii, vv in enumerate(v["baseline_path"]):
            list_json = []
            # with open(vv, "r") as file:
            #     json2 = file.read()
            #     list_json.append(json2)
            json2 = v["baseline_path"]#open(v["baseline_path"])
            print(json2)
    baseline_policies.append(list_json)

iot_all = iot_user.iot_comparison(
    policies=policies,
    baseline_policies=baseline_policies,
    mtr_smoother="kreg",  # this is kreg or HSV
    labels=labels,
    years=years,
    data="CPS"
)

In [ ]:
iot_all.iot[-1].mtr

In [ ]:
iot_all.iot[-2].mtr

In [ ]:
len(iot_all.iot)

In [ ]:
# Plots of f(z) for each year/candidate
fplot = iot_all.plot(var="f")
fplot.update_layout(
    template=template,
)
fplot.write_image(
            os.path.join(path, "income_dist.png")
        )

In [ ]:
# Plots of theta(z) for each year/candidate
theta_plot = iot_all.plot(var="theta_z")
theta_plot.update_layout(
    template=template,
)
theta_plot.write_image(
            os.path.join(path, "theta.png"),
            scale=4
        )

In [ ]:
# Plots of MTRs for each year/candidate
mtr_plot = iot_all.plot(var="mtr")
mtr_plot.update_layout(
    template=template,
)
mtr_plot.update_traces(
    line=dict(dash="dot", color="blue"),
    selector=dict(name="Obama 2015")
)
mtr_plot.update_traces(
    line=dict(dash="dot", color="red"),
    selector=dict(name="Romney 2012")
)
mtr_plot.update_traces(
    line=dict(dash="dash", color="blue"),
    selector=dict(name="Clinton 2016")
)
mtr_plot.update_traces(
    line=dict(dash="dash", color="red"),
    selector=dict(name="Trump 2016")
)
mtr_plot.update_traces(
    line=dict(dash="dashdot", color="blue"),
    selector=dict(name="Biden 2020")
)
mtr_plot.update_traces(
    line=dict(dash="dashdot", color="red"),
    selector=dict(name="Trump 2020")
)
mtr_plot.update_traces(
    line=dict(dash="solid", color="blue"),
    selector=dict(name="Harris 2024")
)
mtr_plot.update_traces(
    line=dict(dash="solid", color="red"),
    selector=dict(name="Trump 2024")
)
mtr_plot.write_image(
            os.path.join(path, "mtrs_all.png"),
            scale=4
        )

In [ ]:
# plots of g(z) for each year/candidate
gz_plot = iot_all.plot(var="g_z")
gz_plot.update_layout(
    template=template,
)
gz_plot.update_traces(
    line=dict(dash="dot", color="blue"),
    selector=dict(name="Obama 2015")
)
gz_plot.update_traces(
    line=dict(dash="dot", color="red"),
    selector=dict(name="Romney 2012")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="blue"),
    selector=dict(name="Clinton 2016")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="red"),
    selector=dict(name="Trump 2016")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="blue"),
    selector=dict(name="Biden 2020")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="red"),
    selector=dict(name="Trump 2020")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="blue"),
    selector=dict(name="Harris 2024")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="red"),
    selector=dict(name="Trump 2024")
)
# gz_plot.update_xaxes(range=[0, 850000])

gz_plot.write_image(
            os.path.join(path, "gz_all.png"),
            scale=4
        )

In [ ]:
# plots of g(z) for each year/candidate, numerical approach
gz_plot = iot_all.plot(var="g_z_numerical")
gz_plot.update_layout(
    template=template,
)
gz_plot.update_traces(
    line=dict(dash="dot", color="blue"),
    selector=dict(name="Obama 2015")
)
gz_plot.update_traces(
    line=dict(dash="dot", color="red"),
    selector=dict(name="Romney 2012")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="blue"),
    selector=dict(name="Clinton 2016")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="red"),
    selector=dict(name="Trump 2016")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="blue"),
    selector=dict(name="Biden 2020")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="red"),
    selector=dict(name="Trump 2020")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="blue"),
    selector=dict(name="Harris 2024")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="red"),
    selector=dict(name="Trump 2024")
)
# gz_plot.update_xaxes(range=[0, 850000])

gz_plot.write_image(
            os.path.join(path, "gz_numerical_all.png"),
            scale=4
        )

# Make the same plot but stop at 200_000
gz_plot.update_xaxes(range=[0, 200_000])
gz_plot.write_image(
            os.path.join(path, "gz_numerical_all_200k.png"),
            scale=4
        )
gz_plot.update_xaxes(range=[0, 100_000])
gz_plot.write_image(
            os.path.join(path, "gz_numerical_all_100k.png"),
            scale=4
        )

In [ ]:
# plots of g(z) for each year/candidate, numerical approach,
# HIGHlighting the Democrats
gz_plot = iot_all.plot(var="g_z_numerical")
gz_plot.update_layout(
    template=template,
)
gz_plot.update_traces(
    line=dict(dash="dot", color="blue"),
    selector=dict(name="Obama 2015")
)
gz_plot.update_traces(
    line=dict(dash="dot", color="gray"),
    selector=dict(name="Romney 2012")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="blue"),
    selector=dict(name="Clinton 2016")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="gray"),
    selector=dict(name="Trump 2016")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="blue"),
    selector=dict(name="Biden 2020")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="gray"),
    selector=dict(name="Trump 2020")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="blue"),
    selector=dict(name="Harris 2024")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="gray"),
    selector=dict(name="Trump 2024")
)
# gz_plot.update_xaxes(range=[0, 850000])

gz_plot.write_image(
            os.path.join(path, "gz_numerical_democrats.png"),
            scale=4
        )
gz_plot.update_xaxes(range=[0, 200_000])
gz_plot.write_image(
            os.path.join(path, "gz_numerical_democrats_200k.png"),
            scale=4
        )
gz_plot.update_xaxes(range=[0, 100_000])
gz_plot.write_image(
            os.path.join(path, "gz_numerical_democrats_100k.png"),
            scale=4
        )

In [ ]:
# plots of g(z) for each year/candidate, numerical approach
# HIGHlighting the Republicans
gz_plot = iot_all.plot(var="g_z_numerical")
gz_plot.update_layout(
    template=template,
)
gz_plot.update_traces(
    line=dict(dash="dot", color="gray"),
    selector=dict(name="Obama 2015")
)
gz_plot.update_traces(
    line=dict(dash="dot", color="red"),
    selector=dict(name="Romney 2012")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="gray"),
    selector=dict(name="Clinton 2016")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="red"),
    selector=dict(name="Trump 2016")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="gray"),
    selector=dict(name="Biden 2020")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="red"),
    selector=dict(name="Trump 2020")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="gray"),
    selector=dict(name="Harris 2024")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="red"),
    selector=dict(name="Trump 2024")
)
# gz_plot.update_xaxes(range=[0, 850000])

gz_plot.write_image(
            os.path.join(path, "gz_numerical_republicans.png"),
            scale=4
        )
gz_plot.update_xaxes(range=[0, 200_000])
gz_plot.write_image(
            os.path.join(path, "gz_numerical_republicans_200k.png"),
            scale=4
        )
gz_plot.update_xaxes(range=[0, 100_000])
gz_plot.write_image(
            os.path.join(path, "gz_numerical_republicans_100k.png"),
            scale=4
        )

In [ ]:
# Show how MTRs vs tax base elasticity affecting g(z) for 2 candidates (separate plots,
# which will be put in a 2 panel figure)
fig = iot_all.JJZFig4(policy='Biden 2020')
fig.update_layout(
    template=template,
)
# fig.update_xaxes(range=[0, 850000])

fig.write_image(
            os.path.join(path, "composition_Biden2020_gz.png"),
            scale=4
        )

In [ ]:
# Loop over values of epsilon and plot Biden under these alternative values"
eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]
biden_eps_dict = {}
biden_eps_dict_numerical = {}
label_list = []
for i, v in enumerate(eps_values):
    label = r"$\varepsilon$ = " + str(v)
    iot_b = iot_user.iot_comparison(
        policies=[candidate_dict["Biden 2020"]["policy_path"]],
        baseline_policies=[candidate_dict["Biden 2020"]["baseline_path"]],
        labels=[label],
        years=[candidate_dict["Biden 2020"]["start_year"]],
        eti=v,
        data="CPS"
    )
    label_list.append(label)
    biden_eps_dict[v] = iot_b.iot[0].df().g_z
    biden_eps_dict_numerical[v] = iot_b.iot[0].df().g_z_numerical

In [ ]:
# plot each g_z
label_dict = {}
redVSblue = n_colors('rgb(0, 0, 255)', 'rgb(255, 0, 0)', len(label_list), colortype = 'rgb')
for i, v in enumerate(label_list):
    label_dict["wide_variable_" + str(i)] = str(eps_values[i])#v
fig = px.line(
    x=iot_b.iot[0].df().z,
    y=[
        biden_eps_dict[0.2],
        biden_eps_dict[0.3],
        biden_eps_dict[0.4],
        biden_eps_dict[0.5],
        biden_eps_dict[0.6]
        ],
    color_discrete_sequence=redVSblue,
    labels=label_dict)
fig.for_each_trace(lambda t: t.update(name = label_dict[t.name], legendgroup = label_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, label_dict[t.name])))
fig.update_layout(
    template=template,
    xaxis_title="Wages and Salaries",
    yaxis_title=r"$g_z$",
    legend=dict(
        title="ETI value:",
        ),
)

# fig.update_xaxes(range=[0, 850000])
fig.write_image(
            os.path.join(path, "vary_ETI_Biden2020_gz.png"),
            scale=4
        )

# plot each g_z_numerical
label_dict = {}
redVSblue = n_colors('rgb(0, 0, 255)', 'rgb(255, 0, 0)', len(label_list), colortype = 'rgb')
for i, v in enumerate(label_list):
    label_dict["wide_variable_" + str(i)] = str(eps_values[i])#v
fig = px.line(
    x=iot_b.iot[0].df().z[50:],
    y=[
        biden_eps_dict_numerical[0.2][50:],
        biden_eps_dict_numerical[0.3][50:],
        biden_eps_dict_numerical[0.4][50:],
        biden_eps_dict_numerical[0.5][50:],
        biden_eps_dict_numerical[0.6][50:]
        ],
    color_discrete_sequence=redVSblue,
    labels=label_dict)
fig.for_each_trace(lambda t: t.update(name = label_dict[t.name], legendgroup = label_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, label_dict[t.name])))
fig.update_layout(
    template=template,
    xaxis_title="Wages and Salaries",
    yaxis_title=r"$g_z$",
    legend=dict(
        title="ETI value:",
        ),
)

# fig.update_xaxes(range=[0, 850000])
fig.write_image(
            os.path.join(path, "vary_ETI_Biden2020_gz_numerical.png"),
            scale=4
        )


In [ ]:
# Redo above with varying epsilon(z) according to some empirical studies
# Required modification of model
eti_dict = {
    "eti_values": [0.18, 0.106, 0.567, 1.83, 1.9],
    "knot_points": [30000, 75000, 250000, 2000000, 10000000]
}
iot_all_vary = iot_user.iot_comparison(
    policies=policies,
    baseline_policies=baseline_policies,
    labels=labels,
    years=years,
    eti=eti_dict,
    data="CPS"
)
# plots of g(z) for each year/candidate, numerical approach
gz_plot = iot_all_vary.plot(var="g_z_numerical")
gz_plot.update_layout(
    template=template,
)
gz_plot.update_traces(
    line=dict(dash="dot", color="blue"),
    selector=dict(name="Obama 2015")
)
gz_plot.update_traces(
    line=dict(dash="dot", color="red"),
    selector=dict(name="Romney 2012")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="blue"),
    selector=dict(name="Clinton 2016")
)
gz_plot.update_traces(
    line=dict(dash="dash", color="red"),
    selector=dict(name="Trump 2016")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="blue"),
    selector=dict(name="Biden 2020")
)
gz_plot.update_traces(
    line=dict(dash="dashdot", color="red"),
    selector=dict(name="Trump 2020")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="blue"),
    selector=dict(name="Harris 2024")
)
gz_plot.update_traces(
    line=dict(dash="solid", color="red"),
    selector=dict(name="Trump 2024")
)
# gz_plot.update_xaxes(range=[0, 850000])

gz_plot.write_image(
            os.path.join(path, "gz_numerical_all_vary_eti.png"),
            scale=4
        )

In [ ]:
# Plot how ETI varies with income
z_line = np.linspace(1, 1000000, 100000)
eti_dict = {
    "eti_values": [0.18, 0.106, 0.567, 1.83, 1.9],
    "knot_points": [30000, 75000, 250000, 2000000, 10000000]
}
eti_spl = UnivariateSpline(
    eti_dict["knot_points"], eti_dict["eti_values"], k=3, s=0
)
eti = eti_spl(z_line)
fig = px.line(x=z_line, y=eti, labels={"x": "Wages and Salaries", "y": r"$\varepsilon$"})
# add special markers without hoverinfo
fig.add_traces(
    go.Scatter(
        x=eti_dict["knot_points"][:-2], y=eti_dict["eti_values"][:-2], mode="markers", name="Gruber and Saez (2022)", hoverinfo="skip"
    )
)
# put legend at bottom
fig.update_layout(legend=dict(yanchor="bottom", y=0.7, xanchor="left", x=0.1))
fig.update_layout(
    template=template,
)
fig.write_image(
            os.path.join(path, "ETI_spline.png"),
            scale=4,
        )

In [ ]:
# Do experiment where hold constant g(z) - pick a candidate as baseline - then plot epsilon(z)
# that would recover those g(z) given the tax rates of each candidate
# one plot with epsilon(z) for each candidate
# Will pick Trump and Clinton (2016) for example

# First, plot just their g(z)
fig = px.line(
    x=iot_all.iot[2].df().z[10:],
    y=[iot_all.iot[2].df().g_z_numerical[10:], iot_all.iot[3].df().g_z_numerical[10:]],
    labels={"x": "Wages and Salaries", "y": r"$g_z$"},
    )
fig.update_layout(
    template=template,
    legend=dict(
        title="Candidate:",
        ),
)
candidate_name = ["Obama 2012", "Romney 2012"]
label_dict = {}
for i, v in enumerate(candidate_name):
    label_dict["wide_variable_" + str(i)] = str(candidate_name[i])
fig.for_each_trace(lambda t: t.update(name = label_dict[t.name], legendgroup = label_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, label_dict[t.name])))

fig.write_image(
            os.path.join(path, "romney_obama_g_z_numerical.png"),
            scale=4
        )
candidate_name = ["Clinton 2016", "Trump 2016"]
label_dict = {}
for i, v in enumerate(candidate_name):
    label_dict["wide_variable_" + str(i)] = str(candidate_name[i])
fig.for_each_trace(lambda t: t.update(name = label_dict[t.name], legendgroup = label_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, label_dict[t.name])))

fig.write_image(
            os.path.join(path, "trump_clinton_g_z_numerical.png"),
            scale=4
        )
candidate_name = ["Biden 2020", "Trump 2020"]
label_dict = {}
for i, v in enumerate(candidate_name):
    label_dict["wide_variable_" + str(i)] = str(candidate_name[i])
fig.for_each_trace(lambda t: t.update(name = label_dict[t.name], legendgroup = label_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, label_dict[t.name])))

fig.write_image(
            os.path.join(path, "trump_biden_g_z_numerical.png"),
            scale=4
        )
candidate_name = ["Harris 2024", "Trump 2024"]
label_dict = {}
for i, v in enumerate(candidate_name):
    label_dict["wide_variable_" + str(i)] = str(candidate_name[i])
fig.for_each_trace(lambda t: t.update(name = label_dict[t.name], legendgroup = label_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, label_dict[t.name])))

fig.write_image(
            os.path.join(path, "trump_harris_g_z_numerical.png"),
            scale=4
        )
# Now find the epsilon(z) that would give Trump's policies the same g(z) as Clinton
eti_beliefs_lw, eti_beliefs_jjz = iot.inverse_optimal_tax.find_eti(iot_all.iot[2], iot_all.iot[3], g_z_type="g_z")
# idx = (np.where(np.absolute(eti_beliefs_jjz[1:]) < 1))[0]
idx = (np.where(((iot_all.iot[3].df().g_z.values) >  1.05) | ((iot_all.iot[3].df().g_z.values) < 0.95)))[0]
fig2 = px.line(
    x=iot_all.iot[2].df().z[idx],
    y=eti_beliefs_jjz[idx],
    labels={"x": "Wages and Salaries", "y": r"$\text{Implied } \varepsilon$"},
    )
fig2.update_layout(
    template=template,
)
fig2.write_image(
            os.path.join(path, "trump_eti.png"),
            scale=4
        )

In [ ]:
(np.where(((iot_all.iot[2].df().z.values) >  1.01) | ((iot_all.iot[2].df().z.values) < 0.99)))[0]

In [ ]:
eti_dict = {
    "eti_values": [0.18, 0.106, 0.567, 1.83, 1.9],
    "knot_points": [30000, 75000, 250000, 2000000, 10000000]
}
# ETI values from Gruber and Saez (2002) (Table 3) and Saez (2004) (Tables 2, 4, 5)
# Compute MTR schedule under current law
iot_2023 = iot_user.iot_comparison(
        policies=[{}],
        baseline_policies=[None],
        labels=["2023 Law"],
        years=[2023],
        data="CPS",
        eti=eti_dict
    )
fig = px.line(
    x=iot_2023.iot[0].df().z,
    y=iot_2023.iot[0].df().mtr
    )
fig.update_layout(
    template=template,
    xaxis_title="Wages and Salaries",
    yaxis_title=r"$T'(z)$",
)
fig.write_image(
            os.path.join(path, "MTR_2023.png"),
            scale=4
        )